<a href="https://colab.research.google.com/github/dmitriygorlov/Yandex.Practikum_Data_Science/blob/main/Module-03_01-Linear-algebra/project-9_linear-algebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Защита персональных данных клиентов

Нужно разработать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, но качество моделей машинного обучения не ухудшилось.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

загрузили библиотеки

In [2]:
df = pd.read_csv("https://code.s3.yandex.net/datasets/insurance.csv")

загрузили данные

In [3]:
df.info()
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


наши исходные данные состоят из 5000 строк, есть целевой признак "Страховые выплаты", а также признаки пол, возраст, зарплата, члены семьи. Поделим данные на features и target, а также на тренировочный и тестовый набор.

In [4]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

посчитаем r2 на изначальной регрессии

In [5]:
model = LinearRegression()
model.fit(features_train, target_train)
r2 = model.score(features_test, target_test)
print('r2 =', r2)

r2 = 0.435227571270266


Итак, мы знаем исходное значение качества на модели, теперь приступим к умножению матриц и исследуем влияние.

## Умножение матриц

Создадим случайную матрицу 4 на 4 и проверим её на обратимость перед умножением

In [6]:
matrix = np.random.rand(4, 4)

ver_matrix = np.linalg.inv(matrix)

подходящая обратимая матрица готова, умножим исходные данные и оценим изменение качества модели (r2)

In [7]:
features_train_mat = features_train @ matrix
features_test_mat = features_test @ matrix


model_mat = LinearRegression()
model_mat.fit(features_train_mat, target_train)
r2 = model_mat.score(features_test_mat, target_test)
print('r2 =', r2)

r2 = 0.4352275712700624


**Ответ:** Метрика r2 по сути не изменилась, а значит качество линейной регрессии не меняется при умножении признаков на обратимую матрицу. Это происходит вввиду самой логики формулы линейной регрессии. Посмотрим подробнее

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Важные свойства матриц:

$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
(AB)^{T} = B^{T} A^{T}
$$
$$
A A^{-1} = E
$$


**Обоснование:**

Заменим матрицу $X$ на $Y$

$$
N = XO
$$

где $O$ - обратимая матрица со случайными значениями

После замены посмотрим на новые $w_{1}$ и $a_{1}$

$$
w_{1}=(N^T N)^{-1} N^T y
$$

$$
a_{1}=N (N^T N)^{-1} N^T y
$$

Раскроем N

$$
a_{1}=XO ((XO)^T (XO))^{-1} (XO)^T y
$$

Теперь воспользуемся свойствами матриц

$$
XO ((XO)^T (XO))^{-1} (XO)^T y = XO(XO)^{-1}((XO)^T)^{-1}(XO)^Ty = XOO^{-1}X^{-1}((XO)^T)^{-1}(XO)^Ty = XEX^{-1}((XO)^T)^{-1}(XO)^Ty = XX^{-1}(O^T X^T)^{-1} O^T X^T y = X X^{-1}(X^T)^{-1}(O^T)^{-1}O^T X^T y = X X^{-1}(X^T)^{-1}EX^T y = X(X^T X)^{-1} X^T y
$$

таким образом мы получили исходное 

$$
a = Xw = X(X^T X)^{-1} X^T y
$$

а это значит, что $a_{1}$ = $a$ и значение предсказания не меняется, если умножать матрицу признаков на обратимую матрицу!

## Алгоритм преобразования

**Алгоритм**

Для шифрования исходных данных сделаем функцию, которая будет создавать подходящую матрицу и перемножать исходные данные на неё

In [8]:
def algoritm(features_train, features_test):
    n = features_train.shape[1]
    matrix = np.random.rand(n, n)
    det = np.linalg.det(matrix)
    while det == 0:
        matrix = np.random.rand(n, n)
        det = np.linalg.det(matrix)
    new_features_train = features_train @ matrix
    new_features_test = features_test @ matrix
    return new_features_train, new_features_test

**Обоснование**

Как мы уже доказали ранее, качество линейной регрессии не изменяется при умножении признаков на обратимую матрицу. Поэтому в рамках функции мы берём исходные данные, узнаем подходящую размерность, создаём подходящую матрицу, проверяем матрицу на обратимость (при необходимости создаём новую) и возвращаем перемноженные матрицы. Проверим алгоритм на наших данных

## Проверка алгоритма

Исходные данные:

In [9]:
model = LinearRegression()
model.fit(features_train, target_train)
r2 = model.score(features_test, target_test)
print('r2 =', r2)

r2 = 0.435227571270266


Зашфруем данные и померяем r2 снова:

In [10]:
cipher_features_train, cipher_features_test = algoritm(features_train, features_test)

model_cipher = LinearRegression()
model_cipher.fit(cipher_features_train, target_train)
cipher_r2 = model_cipher.score(cipher_features_test, target_test)
print('cipher_r2 =', cipher_r2)

cipher_r2 = 0.43522757126978073


Метрика качества (r2) не поменялась! 

**Вывод:** функцию алгоритма можно применять для шифровки исходных данных без падения качества предсказания модели